# Hello World
This notebook shows how to create an email and send it.

In [1]:
import nuclio

In [2]:
# nuclio: start-code
%nuclio config kind = "job"
%nuclio config spec.image = "mlrun/ml-models"

%nuclio: setting kind to 'job'
%nuclio: setting spec.image to 'mlrun/ml-models'


In [3]:
from mlrun.execution import MLClientCtx

# Import the modules we'll need
from datetime import datetime



# For guessing MIME type based on file name extension
import mimetypes

def hello_world(
    context: MLClientCtx   
) -> None:
    """Log Hello World"""
    # datetime object containing current date and time
    now = datetime.now()
    # dd/mm/YY H:M:S
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    # Validate inputs
    context.logger.info("Hello Iguazzzzzzzzzzzzzzzz, the time now is {}".format(dt_string))

In [4]:
# nuclio: end-code

## MLRun conf

In [5]:
from mlrun import mlconf
import os

#artifact_path = mlconf.artifact_path or os.path.abspath('jobs')
artifact_path = os.path.abspath('jobs')
mlconf.dbpath = 'http://mlrun-api:8080'
mlconf.artifact_path = artifact_path
print(f'Artifacts path: {mlconf.artifact_path}\nMLRun DB path: {mlconf.dbpath}')

Artifacts path: /User/functions/hello_world/jobs
MLRun DB path: http://mlrun-api:8080


## Save function

In [6]:
from mlrun import code_to_function

# create job function object from notebook code
fn = code_to_function("hello_world")
# add metadata (for templates and reuse)
fn.spec.default_handler = "hello_world"
fn.spec.description = "mlrun hello world"
fn.export("function.yaml")

> 2020-07-16 09:49:20,443 [info] function spec saved to path: function.yaml


# Test the function
First, configure MLRun. Define project parameters that will be used for testing the function

In [7]:
from os import path, getenv
from mlrun import new_project

project_name = '-'.join(filter(None, ['hello-world', getenv('V3IO_USERNAME', None)]))
project_path = path.abspath('conf')
project = new_project(project_name, project_path, init_git=True)

print(f'Project path: {project_path}\nProject name: {project_name}')

Project path: /User/functions/hello_world/conf
Project name: hello-world-admin


In [8]:
from mlrun import run_local, NewTask, import_function, mount_v3io

# Target location for storing pipeline artifacts
artifact_path = path.abspath('jobs')
# MLRun DB path or API service URL
mlconf.dbpath = mlconf.dbpath or 'http://mlrun-api:8080'

print(f'Artifacts path: {artifact_path}\nMLRun DB path: {mlconf.dbpath}')

Artifacts path: /User/functions/hello_world/jobs
MLRun DB path: http://mlrun-api:8080


## Configure task parameters to be used when executing the function

***Make sure to replace placeholders with actual SMTP configuration (address/email/password)***

In [9]:
task_params = { }

task = NewTask(name='hello_task', project=project_name, handler=hello_world, artifact_path=artifact_path,
              params=task_params)


## Run locally

In [10]:
hello_workd_run = run_local(task,name='hello_world')

> 2020-07-16 09:49:20,646 [info] starting run hello_world uid=6ad8c099acde47c5b0924d716db2ac69  -> http://mlrun-api:8080
> 2020-07-16 09:49:20,787 [info] Hello Iguazzzzzzzzzzzzzzzz, the time now is 16/07/2020 09:49:20


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
hello-world-admin,...6db2ac69,0,Jul 16 09:49:20,completed,hello_world,v3io_user=adminkind=handlerowner=adminhost=jupyter-b84b776f6-2s8ts,,,,


to track results use .show() or .logs() or in CLI: 
!mlrun get run 6ad8c099acde47c5b0924d716db2ac69 --project hello-world-admin , !mlrun logs 6ad8c099acde47c5b0924d716db2ac69 --project hello-world-admin
> 2020-07-16 09:49:20,898 [info] run executed, status=completed


## Run remotely

In [11]:
# Convert the local get_data function into an email_func project function
hello_func = code_to_function(name='hello_world')
hello_func.apply(mount_v3io())

hello_func.run(task, params=task_params,  workdir=mlconf.artifact_path, schedule="* * * * *")
hello_func.doc()

> 2020-07-16 09:49:24,128 [info] starting run hello_task uid=0adc4b3690f54abd9ac6dce96f9c2c60  -> http://mlrun-api:8080
> 2020-07-16 09:49:24,297 [info] task scheduled, {'schedule': '* * * * *', 'id': 139742000228880}
function: hello-world

entry points:
  hello_world: Log Hello World
    context(MLClientCtx)  - , default=
